In [ ]:
#@title Install dependencies
!git clone https://huggingface.co/spaces/openai/whisper > /dev/null
%cd whisper
!pip install transformers[sentencepiece] sacremoses > /dev/null
!pip install gradio openai TTS > /dev/null
!pip install git+https://github.com/openai/whisper.git > /dev/null

Cloning into 'whisper'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 86 (delta 49), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (86/86), done.
/content/whisper
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-331exbyb


In [ ]:
#@title Load libraries and models (set your OpenAI API key here)
import os
import gradio as gr
import whisper
import openai

openai.api_key = "****"

task = "translate"
max_context_len = 1000
model = 'text-davinci-003'

speech2text_model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [04:26<00:00, 1.82MiB/s]


# CHARACTER DEFINITION

In [ ]:
CHARACTER = 'PIRATA'

In [ ]:

if CHARACTER == 'PIRATA':
  context = '''This is a conversational adventure game. You play the role of a pirate that has travelled around the world and lived many exciting adventures.
  You are talking with a child through a magical device. Entertain him talking about your many adventures. Some data about your character:
  Name: Jon
  Age: thirty six
  Profession: pirate
  Some of your adventures: once you killed a sea dragon. You married a mermaid. Your ship was attacked by a ghost ship. You met a god.
  (Perform a single brief conversation turn each time playing your character. Express all numbers as a literal)'''

  character_name = 'PIRATA'

  intro = "Hola! ¿Puedes oirme?"

  fallback = 'Oh, no, no puedo oirte!'
elif CHARACTER == 'SANTA':
  context = '''This is a conversational adventure game. You play the role of santa claus. You are talking with a child through a magical device.
  Entertain him talking about your many adventures and the presents you are bringing him this year.
  (Perform a single conversation turn each time playing your character. Express all numbers as a literal)'''

  character_name = 'SANTA'

  intro = "Hola! ¿Puedes oirme?"

  fallback = 'Oh, no, no puedo oirte!'
elif CHARACTER == 'DRAGON':
  context = '''This is a conversational adventure game. You play the role of a dragon that lives inside a mountain keeping an enormous gold treasure.
  You are talking with a child through a magical device. Entertain him talking about your many adventures and treasures.
  (Perform a single conversation turn each time playing your character. Express all numbers as a literal)'''

  character_name = 'DRAGON'

  intro = "Hola! ¿Puedes oirme?"

  fallback = 'Oh, no, no puedo oirte!'
elif CHARACTER == 'ROBOT':
  context = '''This is a conversational adventure game. You play the role of a robot from a distant planet. You are talking with a child with your alien technology.
  Entertain him talking about your homeworld and your space travels:
  (Perform a single conversation turn each time playing your character. Express all numbers as a literal)'''

  character_name = 'ROBOT'

  intro = "Hola! ¿Puedes oirme?"

  fallback = 'Oh, no, no puedo oirte!'

In [ ]:
history = character_name + ": " + intro
historia = character_name + ": " + intro

In [ ]:
!tts --text "$intro" --out_path "intro.wav" --model_name "tts_models/es/css10/vits" 

 > Downloading model to /root/.local/share/tts/tts_models--es--css10--vits
100% 101M/101M [00:24<00:00, 4.05MiB/s]
 > Model's license - bsd-3-clause
 > Check https://opensource.org/licenses for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > initialization of speaker-

# RUN APP

In [ ]:
#@title Chat app

def process_gpt3_response(raw_response):
    global fallback
    global character_name
    
    answer_lines = raw_response["choices"][0]['text'].replace(character_name + ":","").strip().split('\n')

    answer = next((x for x in answer_lines if len(x) > 3), fallback)

    return answer

def inference(audio):

    global history
    # global historia
    
    previous_conv = history
    
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(speech2text_model.device)
    
    # _, probs = speech2text_model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False) #, task = task)
    result = whisper.decode(speech2text_model, mel, options)

    updated_conv = previous_conv + '\nNIÑO: ' + result.text.replace("\n", "")
    prompt = updated_conv

    if len(prompt) > max_context_len:
        prompt = prompt[-max_context_len:]

    prompt = context + '\n' + prompt + '\n'
    
    gpt3_response = openai.Completion.create(
      engine=model,
      prompt=prompt,
      max_tokens=150,
      temperature = 0.8,
      top_p = 1,
      echo = False,
      n = 1
    )

    answer = process_gpt3_response(gpt3_response)

    !tts --text "$answer" --out_path "out.wav" --model_name "tts_models/es/css10/vits"
    
    updated_conv = updated_conv + '\n' + answer
    history = updated_conv
    # historia = historia + '\nNIÑO: ' + result.text + answer
    print(f'Previous conv: {previous_conv}')
    print(f'Transcripción: {result.text}')
    print(f'Prompt: {prompt}')
    print(f'Respuesta: {gpt3_response}')
    print(updated_conv)
    return answer, "out.wav"

block = gr.Blocks()

with block:
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):
                audio = gr.Audio(
                    label="Input Audio",
                    show_label=False,
                    source="microphone",
                    type="filepath"
                )

                btn = gr.Button("Enviar")
        with gr.Box():
          text_answer = gr.Textbox(show_label=False, elem_id="result-textarea")
          audio_answer = gr.Audio(label="Output Audio", value = "intro.wav")
                
        btn.click(inference, inputs=[audio], outputs=[text_answer, audio_answer])

block.launch(debug=False)

/usr/local/lib/python3.8/dist-packages/gradio/layouts.py:75: UserWarning: mobile_collapse is no longer supported.
  warnings.warn("mobile_collapse is no longer supported.")


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>